In [86]:
import tweepy
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import time
from datetime import datetime


consumer_key = "xpDldnAQyAyNBuoK08TiiiOgX"
consumer_secret = "KXxscYkr2tV5e4eqDsvVHz848eeFOFg6aMaYLA9y46PfHbtHtq"
access_token = "948782412078530560-EQLSb1bqkYNa0xVEaR1CddGObN2uNHj"
access_token_secret = "FK7SjBHhXXWCWWmJ7yWat1D0a25ROI1CfI65b54w5R3cK"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

## Vader Sentiment Analysis

In [101]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Target Search Term
#target_terms = ("@OPECnews", "@Shell", "@exxonmobil", "@Chevron", "@conocophillips", "@OXY_Petroleum", 
                #"@ValeroEnergy", "@Phillips66Co", "@MarathonPetroCo", "@DevonEnergy")
target_terms = ('@IvankaTrump', '@BarackObama')

# "Real Person" Filters
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"

# Array to hold sentiment
sentiments = []

# Loop through all target users
for target in target_terms:
    
    # Defining a counter
    counter = 1

    # Loop through 16 times
    for x in range(1):

        # Run search around each tweet
        public_tweets = api.search(target, count=100, result_type="recent")

        # Loop through all tweets
        for tweet in public_tweets["statuses"]:
            #print(tweet['text'])
            
            # Use filters to check if user meets conditions
            if (tweet["user"]["followers_count"] < max_followers and
                    tweet["user"]["statuses_count"] > min_tweets and
                    tweet["user"]["statuses_count"] < max_tweets and
                    tweet["user"]["friends_count"] < max_following and
                    tweet["user"]["lang"] == lang):

                # Run Vader Analysis on each tweet
                compound = analyzer.polarity_scores(tweet["text"])["compound"]
                pos = analyzer.polarity_scores(tweet["text"])["pos"]
                neu = analyzer.polarity_scores(tweet["text"])["neu"]
                neg = analyzer.polarity_scores(tweet["text"])["neg"]
                tweets_ago = counter
                
                # Conversion of actual timestamp
                raw_time = tweet["created_at"]
                #text = tweet["text"]
                converted_time = datetime.strptime(raw_time, "%a %b %d %H:%M:%S %z %Y")
                time_format = converted_time.strftime("%Y-%m-%d")
                #print(text)
                
                # Store all the sentiments
                try:
                    sentiments.append({"Source Account": tweet['entities']['user_mentions'][0]['name'],
                                       "Media Sources": target,
                                       "Tweet Text": text,
                                       "Actual Date": tweet["created_at"],
                                       "Date": time_format,
                                       "Tweet Polarity": compound,
                                       "Positive": pos,
                                       "Neutral": neu,
                                       "Negative": neg,
                                       "Tweets Ago": counter})
                except:
                    continue

                # Add to counter 
                counter = counter + 1

In [102]:
sentiments_pd = pd.DataFrame(sentiments)
sentiments_pd.head()

,Actual Date,Date,Media Sources,Negative,Neutral,Positive,Source Account,Tweet Polarity,Tweet Text,Tweets Ago
0,Fri Jan 19 09:09:42 +0000 2018,2018-01-19,@IvankaTrump,0.00,0.859,0.141,Fox News,0.4215,RT @KibBitzLaw: Current mood: \n\nHappiness wi...,1
1,Fri Jan 19 09:09:15 +0000 2018,2018-01-19,@IvankaTrump,0.35,0.538,0.112,Olga_Lautman NYC ✨,-0.6511,RT @KibBitzLaw: Current mood: \n\nHappiness wi...,2
2,Fri Jan 19 09:07:39 +0000 2018,2018-01-19,@IvankaTrump,0.00,1.000,0.000,Ivanka Trump,0.0000,RT @KibBitzLaw: Current mood: \n\nHappiness wi...,3
3,Fri Jan 19 09:05:49 +0000 2018,2018-01-19,@IvankaTrump,0.00,0.417,0.583,Ivanka Trump,0.9272,RT @KibBitzLaw: Current mood: \n\nHappiness wi...,4
4,Fri Jan 19 09:04:40 +0000 2018,2018-01-19,@IvankaTrump,0.00,0.500,0.500,Lindsey Nolan,0.7184,RT @KibBitzLaw: Current mood: \n\nHappiness wi...,5


In [103]:
sentiments_vader = sentiments_pd['Tweet Polarity'].head(20)
sentiments_vader

0     0.4215
1    -0.6511
2     0.0000
3     0.9272
4     0.7184
5     0.0000
6    -0.0516
7     0.0000
8     0.0000
9     0.5859
10   -0.5574
11    0.6360
12    0.0000
13    0.0000
14    0.0000
15    0.0000
16    0.3818
17    0.3818
18    0.3612
19    0.9260
Name: Tweet Polarity, dtype: float64

## IBM Watson Sentiment Analysis

In [104]:
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions

natural_language_understanding = NaturalLanguageUnderstandingV1(
  username='eead473e-d77e-4ab6-bbcf-48196a0cb3e6',
  password='4e1sOJAWwIIB',
  version='2017-02-27')

sentiment_score = []

for tweet_text in sentiments_pd["Tweet Text"]:
    try:
        response = natural_language_understanding.analyze(
          text = tweet_text,
          features=Features(
            entities=EntitiesOptions(
              emotion=True,
              sentiment=True,
              limit=2),
            keywords=KeywordsOptions(
              emotion=True,
              sentiment=True,
              limit=2)))
        #fix_floats(response)
        tweet_sentiment = json.dumps(response['keywords'][0]['sentiment']['score'], sort_keys=True, indent=4, separators=(',', ': '))
        sentiment_label = json.dumps(response['keywords'][0]['sentiment']['label'], sort_keys=True, indent=4, separators=(',', ': '))
        sentiment_score.append({"Tweet Sentiment": tweet_sentiment,
                            "Sentiment Label": sentiment_label})
    except:
        continue
        
sentiment_final = pd.DataFrame(sentiment_score)
sentiment_watson = sentiment_final['Tweet Sentiment']

In [105]:
sentiment_watson.head(20)

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
Name: Tweet Sentiment, dtype: object

## Converting 'String' to 'float' (traverse list or dictionary)

In [61]:
def fix_floats(data):
    if isinstance(data,list):
        iterator = enumerate(data)
    elif isinstance(data,dict):
        iterator = data.items()
    else:
        raise TypeError("can only traverse list or dict")

    for i,value in iterator:
        if isinstance(value,(list,dict)):
            fix_floats(value)
        elif isinstance(value,str):
            try:
                data[i] = float(value)
            except ValueError:
                pass